# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante

### Shinto - Lazio

URL di riferimento: https://www.tripadvisor.it/Restaurant_Review-g187791-d1992187-Reviews-Shinto-Rome_Lazio.html

Home --> Ristoranti --> Firenze --> Best_ranking--> SENZ_ALTRO_Bistrot

In [1]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [2]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g187791-d1992187-Reviews-Shinto-Rome_Lazio.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Done")

Done


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, i commenti, le review e da dove è stato scritto il commento per quanto rigurda la prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [3]:
# OTTENGO UNA LISTA DI DIV CHE CONTENGONO NOME, RECENSIONE E VOTO

all_reviews = []
all_names = []
all_ratings = []
all_mobile = []

review_container = clear_data.findAll("div", attrs={"class":"review-container"})

for container in review_container:
  
    ### REVIEW ###
    review = container.find('p', attrs={'class': 'partial_entry'})
    if review:
        all_reviews.append(review.text)
        
    ### NAME ###
    name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
    if name:
      all_names.append(name.text)
    else:
      all_names.append('NO_NAME')
    
    ### RATING ###
    rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
    all_ratings.append(rating['class'][1][7:8])
    
    ### MOBILE ###
    mobile = container.find('span', attrs={'class': 'viaMobile'})
    if mobile:
      all_mobile.append('Mobile')
    else:
      all_mobile.append('PC')
    
    
################################################################################################################################
        
print("Executed")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
Nomi:  10
Commenti:  10
Voti:  10
Scritto da:  10


### SECONDO SCRAPING (dalla seconda pagina in poi)

Notiamo dall'URL che è possibile iterare sulle pagine, ottenendo i dati!

In [4]:
from tqdm import tqdm

URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g187791-d1992187-Reviews-or10-Shinto-Rome_Lazio.html'

URL_p1 = 'https://www.tripadvisor.it/Restaurant_Review-g187791-d1992187-Reviews-or10-Shinto-Rome_Lazio.html'
URL_p2 = '0-Da_Fortunato_al_Pantheon-Rome_Lazio.html'
    
# OTTENGO I COMMENTI, NOMI E VOTI DALLA SECONDA PAGINA IN POI

total_pages = int(clear_data.select("a.pageNum.last.taLnk")[0].text) # 25 come intero

# per iterare dobbiamo partire dalla seconda pagina dei commenti, così nell'URL compaiono le cifre che ci servono
for r in tqdm(range(1,total_pages)):

    # OTTENGO L'URL DELLA PAGINA
    URL_temp = URL_p1 + str(r) + URL_p2

    # OTTENGO I DATI E CI APPLICO BEATIFULSOUP
    data = requests.get(URL_temp)
    clear_data = BeautifulSoup(data.content, "lxml")
    

    review_container = clear_data.findAll("div", attrs={"class":"review-container"})

    for container in review_container:
  
      ### REVIEW ###
      review = container.find('p', attrs={'class': 'partial_entry'})
      if review:
          all_reviews.append(review.text)

      ### NAME ###
      name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
      if name:
        all_names.append(name.text)
      else:
        all_names.append('NO_NAME')

      ### RATING ###
      rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
      all_ratings.append(rating['class'][1][7:8])

      ### MOBILE ###
      mobile = container.find('span', attrs={'class': 'viaMobile'})
      if mobile:
        all_mobile.append('Mobile')
      else:
        all_mobile.append('PC')
    
        
################################################################################################################################
        
print("Executed")
print('total_pages: ', total_pages)
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_mobile))

100%|██████████| 89/89 [02:11<00:00,  1.54s/it]

Executed
total_pages:  90
Nomi:  895
Commenti:  895
Voti:  895
Scritto da:  895


In [0]:
# display(all_reviews)

In [0]:
# print(all_names)

In [0]:
#print(all_ratings, '\n')
#print(len(all_ratings))

In [0]:
#display(all_mobile)

#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [9]:
all_reviews_filtered = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        all_reviews_filtered.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        all_reviews_filtered.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate, '\n')

display(all_reviews_filtered[:10])

Totale recensioni: 895 
 Recensioni analizzabili: 626 
 Recensioni scartate: 269 



['Sempre un ottimo ristorante con piatti ben cucinati e con un sevizio tranquillo e non sgangherato come sovente avviene, Prezzo corretto ed insomma una sicurezza da apprezzare.',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'Riprovato dopo molto tempo.\nCarciofo tenero.\nRigatone all’amatriciana buono ma porzione troppo ridotta.\nVino al calice buono.\nServizio cortese ma un po’ freddo.\nServizi puliti.\nPrezzo medio.\nMale sulle risposte intimidatorie alle recensioni critiche: il buon ristoratore non lo fa.',
 'Dopo oltre un anno sono tornato a passare una serata in questo storico ristorante della città. Solita certezza. Cibo di qualità, servizio elegante e preciso come ben ricordavo. Da ritornare. Lo Consiglio.',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA']

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [10]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto fino a quell'indice
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['maurowave', 'NDB48', 'GiacomoB277', 'Pascal V', 'michelet49', 'cardinedoardo', '219marcellom', '104serenal', 'cavune49', 'ilariag495', 'Giuliano V', 'Eirenector', 'senpessina', 'Ignazio R', '59robertam', 'lukyCT', 'Edoardosala97', 'Marina M', 'Dario M', 'MarrMar', '73GABRY73', 'Andrea D', 'Q1094PRfrancescom', 'AnnaT219', 'Enrica C', 'Bebybeby', 'simoneroma2018', 'ssofias00', 'Lorenzo M', 'Giove67', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'Ciccioerpiccoletto', 'Roberta P', 'Gid22', 'giob259', 'mimino m', 'stefan6655', 'manci78', 'rodolfomeloni', 'RiccardoM2219', 'Giovanni D', 'Stelios P', 'giuseppem1959', 'massimocH6700CR', 'marticpp', 'marcogastone', 'giorgiom956', 'Zeno P', 'Chiarav5', 'Stefano Andrea G', '278Pino', 'klaus67', 'Paolo A', 'Vincenzo V', '260616AB', 'Marco D', 'riccardomarins', 'valeria123c', 'alcatraz79', 'Francesco B', 'lucia1378', 'fly69ch', 'Ntinavale', 'anaidedacosta', 'angelraspagliosi', 'danie

#### Utilizzo gender_guesser
Infine, posso utilizzare la libreria gender_guesser.

NB: Genderize ti permette di analizzare 1000 nomi al giorno.

In [11]:
!pip install gender_guesser

In [12]:
import gender_guesser.detector as gender
d = gender.Detector()

names_with_gender = []

for name in all_names_filtered:
    temp_gender = d.get_gender(name)
    names_with_gender.append(temp_gender)
  
print(names_with_gender)

['unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'female', 'male', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'male', 'unknown', 'male', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', '

In [13]:
all_genders = []
i = 0

for guess in names_with_gender:
    if guess == 'male' or guess == 'mostly_male':
        all_genders.append('M')
    elif guess == 'female' or guess == 'mostly_female':
        all_genders.append('F')
    else:
        all_genders.append('UNKNOWN')

    
print(all_genders)

['UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'M', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'M', 'M', 'UNKNOWN', 'M', 'M', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'M', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN',

---
## Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



### Controlliamo che ci siano lo stesso numero di dati
Potremo così creare una tabella sfruttando Pandas

In [14]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews_filtered))
print('Voti: ', len(all_ratings))
print('Mobile: ', len(all_mobile))

Nomi:  895
Commenti:  895
Voti:  895
Mobile:  895


### Eliminiamo le recensioni troppo lunghe e quelle di cui non sappiamo il genere del commentatore
Creo una lista di liste [ ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ... ]

In [15]:
complete_list = []

i = len(all_names_filtered)

for indice in range(i):
  if all_genders[indice] != 'UNKNOWN' and all_reviews_filtered[indice] != 'RECENSIONE_TROPPO_LUNGA':
    temp = [all_names_filtered[indice], all_genders[indice], all_reviews_filtered[indice], all_ratings[indice], all_mobile[indice]]
    complete_list.append(temp)
    
print('DATI ANALIZZABILI: ', len(complete_list))
display(complete_list[:5])


DATI ANALIZZABILI:  114


[['Giuliano',
  'M',
  'Sono stato mercoledì 7 novembre a mangiare con la mia ragazza in questo rinomato ristorante. Prezzi leggermente alti ma in linea con la posizione, il servizio e la qualità del cibo. Molto buone tutte le portate in particolare la amatriciana e il semifreddo. Consiglio vivamente',
  '4',
  'Mobile'],
 ['Dario',
  'M',
  'Ottima la carbonara e la gricia!! Personale gentile e disponibile nonostante le tantissime persone da servire!! Ci si sente come a casa. Prezzi ottimi per la qualità e anche perché la pasta è tutta fatta in casa',
  '5',
  'Mobile'],
 ['Enrica',
  'F',
  'Qualità degli alcolici (vini, rum, limoncello) medrioche, stesso livello se non inferiore degli altri locali in centro. Prezzi assolutamente sproporzionati per non dire "da furto". Personale maleducato e per niente socievole. Poveri turisti! I romani furbi non ci entreranno mai.Più',
  '1',
  'Mobile'],
 ['Stelios',
  'M',
  'Ambiente carino ma 75 euro per 40 gr di pasta pepe e cacio, fredda 2 ca

### Creiamo la tabella

In [0]:
# !pip install pandas
# !pip install xlsxwriter
import pandas as pd

In [0]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nomi': [elemento[0] for elemento in complete_list], 
                   'Genere': [elemento[1] for elemento in complete_list], 
                   'Recensioni': [elemento[2] for elemento in complete_list],
                   'Voti': [elemento[3] for elemento in complete_list],
                   'Scritta da': [elemento[4] for elemento in complete_list]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('RISTORANTE_18.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [18]:
partial_dataframe = pd.read_excel("RISTORANTE_18.xlsx")

display(partial_dataframe)

,Genere,Nomi,Recensioni,Scritta da,Voti
0,M,Giuliano,Sono stato mercoledì 7 novembre a mangiare con...,Mobile,4
1,M,Dario,Ottima la carbonara e la gricia!! Personale ge...,Mobile,5
2,F,Enrica,"Qualità degli alcolici (vini, rum, limoncello)...",Mobile,1
3,M,Stelios,Ambiente carino ma 75 euro per 40 gr di pasta ...,Mobile,1
4,M,Zeno,"Ambiente accogliente, elegante , vista mozzafi...",Mobile,5
5,M,Paolo,Ristorante consigliatomi all hotel ma rivelato...,Mobile,3
6,M,Saro,Un ristorante troppo caro e scadente come qual...,PC,2
7,F,Angelika,"Cibo fantastico e servizio sofisticato, se pos...",PC,5
8,F,Chiara,L'aperitivo al wine-bar da Fortunato è ben ser...,Mobile,5
9,M,Antonino,ero gia' stato da fortunato. Il ritorno e' con...,Mobile,4
